In [1]:
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import ImageGrid
from PIL import Image

import cv2
import numpy as np
import pandas as pd
import pickle as pkl
import imageio
from tqdm import tqdm
from tracking_utils import *
from VehicleDetection import *
import glob
from video_utils import *

In [2]:
gt_detect_path = '/home/group05/mcv-m6-2022-team5/week5/cam_pred_gt/seq3_*_gt.pkl'

all_gt_detections = []

# load gt
for file_name in sorted(glob.glob(gt_detect_path)):
    with open(file_name,'rb') as openFile:
        gt_detections = pkl.load(openFile)
        # print(gt_detections)
        all_gt_detections.append(gt_detections)

In [23]:
## load all detections to list
path  = '/home/group05/mcv-m6-2022-team5/week5/tracking_results/kalman_tracking_seq3_*_pp.pkl'
detections_list = []

# load data
for file_name in sorted(glob.glob(path)):
    with open(file_name,'rb') as openFile:
        detections = pkl.load(openFile)
        detections_list.append(detections)

In [28]:
detections_list[1][995]

,frame,track,bbox,colour,size,detection,life,initial_position_x,initial_position_y,static
1,Frame 995,2,"[659, 630, 856, 743]","[230, 123, 254]",22261,"Frame 995, TL [659,630], BR [856,743], Confide...",996,743.0,678.0,False
3,Frame 995,28,"[1663, 953, 1748, 1032]","[127, 128, 103]",6715,"Frame 995, TL [1663,953], BR [1748,1032], Conf...",1,1705.0,992.0,False


In [3]:
sequence = 'S03'
cams = [('c010', 8715), ('c011', 8457), ('c012', 5879), ('c013', 0), ('c014', 5042), ('c015', 8492)]

In [4]:
# SHOW GROUND TRUTH
s_gt = True

# SHOW DETECTIONS
s_detections = True

# SHOW TRACKS
s_tracks = False

video_path = '/home/group05/mcv-m6-2022-team5/datasets/aic19-track1-mtmc-train/train/S03/{}/vdo.avi'

In [37]:
scaling = 0.5

ims = []
shapes = []
camId = 0
for cam, timstamp in cams:
    final_video_path = video_path.format(cam)
    vidcap = cv2.VideoCapture(final_video_path)
    num_frames = int(vidcap.get(cv2.CAP_PROP_FRAME_COUNT))

    _, initial_image = vidcap.read()
    shapes.append(initial_image.shape)
    print(initial_image.shape)
    camAuxIms = []
    for i in tqdm(range(num_frames)):
        _, image = vidcap.read()
        currentTime = vidcap.get(cv2.CAP_PROP_POS_MSEC)
        if currentTime > timstamp:
            if str(i) in all_gt_detections[camId]:
                image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                color = (0,255,0)
                image = drawBoxes(image, all_gt_detections[camId][str(i)], color)

                for indexDf, row in detections_list[camId][i].iterrows():
                    color = row['colour']
                    image = drawBoxes(image, [row['detection']], [int(color[0]), int(color[1]), int(color[2])])
            # image = cv2.resize(image, (int(image.shape[1]*scaling), int(image.shape[0]*scaling)), interpolation = cv2.INTER_AREA)
            camAuxIms.append(image)
    ims.append(camAuxIms)
    camId += 1

(1080, 1920, 3)


100%|██████████| 2141/2141 [00:12<00:00, 168.50it/s]


(1920, 2560, 3)


100%|██████████| 2279/2279 [00:25<00:00, 90.22it/s] 


(1920, 2560, 3)


...

100%|██████████| 2415/2415 [00:26<00:00, 91.01it/s] 


(1080, 1920, 3)


100%|██████████| 2332/2332 [00:14<00:00, 165.48it/s]


(1080, 1920, 3)


100%|██████████| 1928/1928 [00:08<00:00, 233.37it/s]


In [38]:

gif_ims = []
lens = [len(vid) for vid in ims]
# for i in tqdm(range(min(lens))):
for i in tqdm(range(550, 700)):
    imageArray = [ims[camInd][i] for camInd in range(len(ims))]
    fig = plt.figure(figsize=(20., 20.))
    grid = ImageGrid(fig, 111,  # similar to subplot(111)
                    nrows_ncols=(2, 3),  # creates 3x2 grid of axes
                    axes_pad=0.1,  # pad between axes in inch.
                    )

    for ax, im in zip(grid, imageArray):
        # Iterating over the grid returns the Axes.
        ax.imshow(im)
        ax.axis('off')
    # plt.savefig('foo.png', bbox_inches='tight')
    fig.canvas.draw()
    fig.tight_layout()
    img = Image.frombytes('RGB', 
        fig.canvas.get_width_height(),fig.canvas.tostring_rgb())
    gif_ims.append(img)
    # gif_ims.append(cv2.cvtColor(cv2.imread('foo.png'), cv2.COLOR_BGR2RGB))
    plt.clf()
    plt.close()



100%|██████████| 150/150 [03:01<00:00,  1.21s/it]


In [39]:
imageio.mimsave('tracking_gt_tracks.gif', (gif_ims), fps=20)